In [1]:
print("Settings.py should be e.g. cnn_flows_prtrain")
#DESIGN PARAMETERS FOR NEURAL NETWORK

VALIDATION_SPLIT = 0.2 #indicated ratio of training to validation data: 0.2 -> 20% VALIDATION data
RANDOMIZED_SEED = 20
SHUFFLE_DATASET = False

TRAINING_PREPARATION = False

BATCH_SIZE = 5
TIME_STEPS = 40

NR_EPOCHS = 50

LR = 0.001
# MOMENTUM = 0.9
print("Hyperparameters defined")
print("Bug with learning rate when sequence length is not the same for each element of the dataset")

Settings.py should be e.g. cnn_flows_prtrain
Hyperparameters defined
Bug with learning rate when sequence length is not the same for each element of the dataset


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import time
import os
import os.path as path
import copy

from gait_analysis import AnnotationsCasia as Annotations
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
from gait_analysis import Composer
from gait_analysis import WeightWatcher
from gait_analysis import AccuracyTracker

import matplotlib.pyplot as plt

from gait_analysis.Models import PretrainConvFlow

print("done")

done


In [3]:
#change configuration in settings.py
c = Config()
c.config['indexing']['grouping'] = 'person_sequence_angle'
composer = Composer()
transformer = composer.compose()
dataset = CasiaDataset(transform=transformer)
dataset_size = len(dataset)
indices = list(range(dataset_size))
print("Dataset size:", len(dataset))
print("Indices size:", len(indices))
split = int(np.floor(VALIDATION_SPLIT * dataset_size))
print("Split:", split)
if SHUFFLE_DATASET:
    np.random.seed(RANDOMIZED_SEED)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]
train_sampler = torch.utils.data.SequentialSampler(train_indices)
test_sampler = torch.utils.data.SequentialSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=1, sampler=test_sampler)

loading configuration  cnn_flows_pretrain
[OK]
Dataset size: 5
Indices size: 5
Split: 1


In [4]:
net = PretrainConvFlow.PretrainConvFlow()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=LR)

learning_rate_array = []
loss_array = []
# monitor = WeightWatcher(net,['conv5','fc1'])
train_acc_tracker = AccuracyTracker([0,1,2])
test_acc_tracker = AccuracyTracker([0,1,2])

cpu


In [5]:
# if TRAINING_PREPARATION:
#     print('Start training...')
#     print("Expected loss with {} different classes and {} data elements: {}".format(3, len(dataset)-split, (len(dataset)-split)*np.log(3)))
#     running_loss = 0.0
#     print("Batch size:", BATCH_SIZE)
#     print("Evaluating first element...")
#     start_time = time.time()
#     i, batch = next(iter(enumerate(train_loader)))
#     inputs, labels = batch
#     #TODO make list disappear

#     data_in_list = [s.to(device) for s in inputs['flows']]
#     labels = labels.to(device)

#     print("--------------------------\nOriginal input:")
#     print("Data in: List length", len(data_in_list), "List element:", data_in_list[0].size())
#     print("Labels:", labels.size())
#     print("--------------------------\nSingle element:")
#     data_element = data_in_list[0]
#     label_element = labels[:,0].long()
#     print("Data element:", data_element.size())
#     print("Label element:", label_element.size())

#     optimizer.zero_grad() 
#     output_element = net(data_element)

#     loss = criterion(output_element,label_element)
#     loss.backward() 
#     optimizer.step()

#     running_loss += loss.data.item()
#     elapsed_time = time.time() - start_time;
#     print("Loss:{}, expected loss:{}".format(running_loss, np.log(3)))
#     print("Time needed:{}s".format(elapsed_time))
#     print("Expected loss for total training data: ", (len(dataset)-split)*np.log(3))
#     print("Expected training time per epoch:{} min".format(elapsed_time* len(train_loader)*TIME_STEPS/BATCH_SIZE/60))
#     print("Estimated total training time:{} hours".format(elapsed_time* len(train_loader)*TIME_STEPS/BATCH_SIZE*NR_EPOCHS/3600))
# print("done")

In [6]:
print("Start training...")
for epoch in range(NR_EPOCHS): 
    start_time = time.time()
#     print("Epoch:", epoch)
    running_loss = 0.0
    running_test_loss = 0.0
    for i, batch in enumerate(train_loader):
        inputs, labels = batch
        data_in_list = [s.to(device) for s in inputs['flows']]
        labels = labels.to(device)
        
#         print("--------------------------\nOriginal input:")
#         print("Data in: List length", len(data_in_list), "List element:", data_in_list[0].size())
#         print("Labels:", labels.size())

        for j, data_element in enumerate(data_in_list):
            if (labels.size(1) != len(data_in_list)):
                print("WARNING, labels and data_element length do not match in Set",i, "element", j)
#                 print("Label element after")
                print(labels[0:len(data_in_list),j].size())
            label_element = labels[0:len(data_in_list),j].long()
#             print("--------------------------\nSingle element:")
#             print("Data element size:", data_element.size())
#             print("Label element size:", label_element.size())
#             print("Data element", data_element)
#             print("Label element", label_element)
#             print("Label element after",label_element)

#             print("Data element", data_element[0][0])
    
            optimizer.zero_grad() 
            output_element = net(data_element)
#             print("Output element",output_element)
#             print("Label element after",label_element)
            loss = criterion(output_element,label_element)
            loss.backward() 
            optimizer.step()
            
            running_loss += loss.data.item()/(len(dataset)*(1-VALIDATION_SPLIT))/TIME_STEPS*BATCH_SIZE
#             print("losses:",loss.data.item(),len(dataset),TIMESTEPS)
#             print("Prediction:", torch.max(output_element,1)[1])
#             print("Labels:", label_element.long())
            prediction = torch.max(output_element,1)[1]
            train_acc_tracker.update_acc(prediction,label_element)
        
    #TEST SET
    for i,batch in enumerate(test_loader):
        inputs, labels = batch
        data_in_list = [s.to(device) for s in inputs['flows']]
        labels = labels.to(device)
        for j, data_element in enumerate(data_in_list):
            if (labels.size(1) != len(data_in_list)):
                print("WARNING, labels and data_element length do not match in Set",i, "element", j)
                print(labels[0:len(data_in_list),j].size())
            label_element = labels[0:len(data_in_list),j].long()

            output_element = net(data_element)
            test_loss = criterion(output_element,label_element)
            running_test_loss += loss.data.item()/(len(dataset)*VALIDATION_SPLIT)/TIME_STEPS

            prediction = torch.max(output_element,1)[1]
            test_acc_tracker.update_acc(prediction,label_element)
        
            
    print("-----------------------------")
    print("Training set:")
    print("Epoch {}: Loss [{},{}], Acc [{},{}] took {}s".format(epoch, running_loss,running_test_loss,train_acc_tracker.get_total_acc(),test_acc_tracker.get_total_acc(), time.time()-start_time))
    print("Accuracy by class [{},{}] Total elements: [{},{}]".format(train_acc_tracker.get_acc(),test_acc_tracker.get_acc(), train_acc_tracker.get_labels_distribution(),test_acc_tracker.get_labels_distribution()))

    train_acc_tracker.update_graph()
    train_acc_tracker.reset_acc()
    train_acc_tracker.update_loss(running_loss)
    train_acc_tracker.update_lr(LR)
    
    test_acc_tracker.update_graph()
    test_acc_tracker.reset_acc()
    test_acc_tracker.update_loss(running_test_loss)
    
#     monitor.update_weights(net,['conv5','fc1'])

#test_all_preds(test_net) 
print('...Training finished')
# plt.plot(loss_array)

Start training...
-----------------------------
Training set:
Epoch 0: Loss [1.3937314804643393,1.3103938102722168], Acc [0.29375,0.3] took 7.518651247024536s
Accuracy by class [[0.0, 1.0, 0.0],[0.0, 1.0, 0.0]] Total elements: [[66, 47, 47],[15, 12, 13]]
-----------------------------
Training set:
Epoch 1: Loss [1.3873262088745832,1.2827631235122683], Acc [0.2,0.3] took 7.593461036682129s
Accuracy by class [[0.0, 0.6808510638297872, 0.0],[0.0, 1.0, 0.0]] Total elements: [[66, 47, 47],[15, 12, 13]]
-----------------------------
Training set:
Epoch 2: Loss [1.3502299170941114,1.0570862293243404], Acc [0.34375,0.475] took 7.567633390426636s
Accuracy by class [[0.015151515151515152, 1.0, 0.14893617021276595],[0.4, 0.5833333333333334, 0.46153846153846156]] Total elements: [[66, 47, 47],[15, 12, 13]]
-----------------------------
Training set:
Epoch 3: Loss [1.630652810446918,1.2218784093856803], Acc [0.44375,0.325] took 7.510992765426636s
Accuracy by class [[0.4393939393939394, 0.2978723404

In [7]:
NR_EPOCHS = 10
LR = 0.0001

In [8]:
net.features

Sequential(
  (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Conv2d(16, 6, kernel_size=(3, 3), stride=(1, 1))
  (13): ReLU()
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [9]:
# torch.save(net.state_dict(), '/mnt/DATA/HIWI/IBT/saved_models/PretrainConvFlow/001_all_seq_ts40.pt')
# acc_tracker.write_to_csv('/mnt/DATA/HIWI/IBT/saved_models/PretrainConvFlow/001_all_seq_ts40/')
# Later:
# loaded_model = PretrainConvFlow.PretrainConvFlow()
# loaded_model.load_state_dict(torch.load('/mnt/DATA/HIWI/IBT/saved_models/PretrainConvFlow/first_draft'))

In [10]:
# import csv
# path = '/mnt/DATA/HIWI/IBT/saved_models/PretrainConvFlow/001_all_seq_ts40/'
# with open(path + "acc_tot.csv", 'w') as file:
#     wr = csv.writer(file)
#     wr.writerow(acc_tracker.get_acc_tot_graph())
# with open(path + "loss.csv", 'w') as file:
#     wr = csv.writer(file)
#     wr.writerow(acc_tracker.get_loss_graph())
# with open(path + "lr.csv", 'w') as file:
#     wr = csv.writer(file)
#     wr.writerow(acc_tracker.get_lr_graph())
# acc = acc_tracker.get_acc_graph()
# for i in range(len(acc)):
#     with open(path + "acc" + str(i) + ".csv", 'w') as file:
#         wr = csv.writer(file)
#         wr.writerow(acc[i])

In [11]:
# Later:
# loaded_model = PretrainConvFlow.PretrainConvFlow()
# loaded_model.load_state_dict(torch.load('/mnt/DATA/HIWI/IBT/saved_models/PretrainConvFlow/first_draft'))